In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
! cp -r /content/drive/My\ Drive/Hackabit/new /content/

In [8]:
! for f in /content/new/*.mp4; do mkdir ${f:0:-4}; ffmpeg -i $f -r 1/1 ${f:0:-4}/p%d.png; done;

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [0]:
from keras.applications import Xception
from keras.models import Model
from keras.layers import Dense, Input
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Nadam
import numpy as np
import matplotlib.pyplot as plt
import os
import keras.preprocessing.image as kimg
from keras import callbacks
import keras.applications as kapp

In [10]:
cnn_base = Xception(input_shape=(240,240,3),weights="imagenet",include_top=False)
cnn_out = GlobalAveragePooling2D()(cnn_base.output)
cnn = Model(input=cnn_base.input, output=cnn_out)
cnn.trainable = False
cnn.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 240, 240, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 119, 119, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 119, 119, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 119, 119, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [70]:
frames = 10
encoded_frames = Input((frames, 2048))
encoded_sequence = LSTM(256)(encoded_frames)
hidden_layer = Dense(output_dim=1024, activation="relu")(encoded_sequence)
outputs = Dense(output_dim=2, activation="softmax")(hidden_layer)
model = Model([encoded_frames], outputs)
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 10, 2048)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               2360320   
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              263168    
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 2050      
Total params: 2,625,538
Trainable params: 2,625,538
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=1024)`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=2)`
  """


In [0]:
optimizer = Nadam(lr=0.002,beta_1=0.9,beta_2=0.999,epsilon=1e-08,schedule_decay=0.004)
model.compile(loss="categorical_crossentropy",optimizer=optimizer,metrics=["categorical_accuracy"])

In [0]:
# def image_generator(path, labels, frames=10, batch_size=32):
#     l = len(os.listdir(path))
#     i = 1
#     batch_x = np.ndarray((batch_size, frames, 240, 240, 3))
#     batch_y = np.ndarray((batch_size, 2))
#     while True:
#         for j in range(batch_size):
#             for k in range(frames):
#                 batch_x[j,k] = kimg.img_to_array(kimg.load_img(os.path.join(path, 'p'+str(i+k)+'.png'), target_size=(240, 240)))/255
#                 batch_y[j] = labels[i+k-1]
#             i += 1

#         if i > l-frames-batch_size:
#             i = 1
#         yield batch_x, batch_y

In [0]:
def load_img(video):
    with open('/content/annotations/' + video + '.txt') as f:
        t = f.readlines()
    t = list(map(lambda x: list(map(lambda y: round(int(y)/25), x.strip().split())), t))
    l = len(os.listdir('/content/new/' + video))
    labels = np.ndarray((l, 2))
    for i in range(l):
        chngd = False
        for j in t:
            if i+1>=j[0] and i+1<=j[1]:
                labels[i] = [0,1]
                chngd = True
                break
            if not chngd:
                labels[i] = [1,0]
    return labels, l

In [0]:
# train_data = kimg.ImageDataGenerator(rescale=1/255).flow_from_directory('/content/new/', classes=[video], target_size = (240,240), shuffle=False)
def cnn_predict(l, video):
    predicted = np.ndarray((l, 2048))
    for i in range(1, l+1):
        img = kapp.inception_v3.preprocess_input(np.expand_dims(kimg.img_to_array(kimg.load_img('/content/new/' + video + '/p'+str(i)+'.png',
                                                                                                target_size=(240, 240))), axis=0))
        predicted[i-1] = cnn.predict(img)
    return predicted

In [0]:
def build_data(labels, l, predicted):
    num_data = l - frames + 1
    data_x, data_y = np.ndarray((num_data, frames, 2048)), np.ndarray((num_data, 2))
    for i in range(num_data):
        for j in range(frames):
            data_x[i][j] = predicted[i+j]
        data_y[i] = labels[i]
    print(data_x.shape, data_y.shape)
    return data_x, data_y

In [99]:
checkpointer = callbacks.ModelCheckpoint(filepath='/content/drive/My Drive/Hackabit/lstm_weights.hdf5')
vids = list(map(lambda x: x[:-4], os.listdir('/content/annotations')))
data_x, data_y = np.ndarray((0, frames, 2048)), np.ndarray((0, 2))

for vid in vids:
    if vid[0]=='.':
        continue
    labels, l = load_img(vid)
    print(vid, l, end=' ')
    if l<frames:
        continue
    predicted = cnn_predict(l, vid)
    x, y = build_data(labels, l, predicted)
    data_x = np.concatenate((data_x, x))
    data_y = np.concatenate((data_y, y))

HC0wnVTbEoY 132 (123, 10, 2048) (123, 2)
KpPOsd5l6k0 71 (62, 10, 2048) (62, 2)
SCkiKtVpxxc 83 (74, 10, 2048) (74, 2)
fRmIS2eWn2Y 23 (14, 10, 2048) (14, 2)
YULESOhr7Rg 50 (41, 10, 2048) (41, 2)
Q7poHYtkNZY 133 (124, 10, 2048) (124, 2)
klkzFNib8Ks 139 (130, 10, 2048) (130, 2)
h-yKQnXJZVc 134 (125, 10, 2048) (125, 2)
bDkjQcQ-rmw 239 (230, 10, 2048) (230, 2)
F22YJZtVGm4 126 (117, 10, 2048) (117, 2)
MsSA9s-WokU 152 (143, 10, 2048) (143, 2)
8JvjJMADF0w 95 (86, 10, 2048) (86, 2)
qdYPvBfjkyI 110 (101, 10, 2048) (101, 2)
as6c4eZSDoE 45 (36, 10, 2048) (36, 2)
c3o6ibJuP_g 29 (20, 10, 2048) (20, 2)
bE39-VcM2M8 212 (203, 10, 2048) (203, 2)
oiSf_mRVqXg 150 (141, 10, 2048) (141, 2)
P13AamsXZyA 183 (174, 10, 2048) (174, 2)
dSB3rx5aNio 62 (53, 10, 2048) (53, 2)
YnNsZD6gDf0 146 (137, 10, 2048) (137, 2)
E8bP__QcTtY 103 (94, 10, 2048) (94, 2)
m4vJDPjFXT4 97 (88, 10, 2048) (88, 2)
uE_qnCCn62M 184 (175, 10, 2048) (175, 2)
5hQapiCQJqs 148 (139, 10, 2048) (139, 2)
5m5uP3QggAY 51 (42, 10, 2048) (42, 2)
TvK3m0w

In [0]:
hist = model.fit(data_x, data_y, epochs=5, callbacks=[checkpointer])
plt.plot(hist.history['loss'])

Epoch 1/5
8748/8748 [==============================] - 10s 1ms/step - loss: nan - categorical_accuracy: 0.5756
Epoch 2/5
8748/8748 [==============================] - 10s 1ms/step - loss: nan - categorical_accuracy: 0.5756
Epoch 3/5
 288/8748 [..............................] - ETA: 9s - loss: nan - categorical_accuracy: 0.5174

In [0]:
data_x = np.nan_to_num(data_x)

In [0]:
! cp -r /content/new/ /content/drive/My\ Drive/Hackabit

In [0]:
np.save('/content/drive/My Drive/Hackabit/data_y', data_y)